In [ ]:
import pandas as pd
import numpy as np
import time as t
import matplotlib.pyplot as plt
%matplotlib inline

<font color = "#CC3D3D"><p>
# Topics
- [Group Aggregation](#Grou-Aggregation)
- [Pivot Table](#Pivot-Table)
- [Apply와 Applymap](#Apply&Applymap)
- [Datetime](#Datetime)

# Groupby Aggregation
<img align="left" src="http://drive.google.com/uc?export=view&id=17lLj-fLLYk6Dxcz7yBIX7bMEl4PAESBB" width=600>

### groupby
- 데이터를 그룹별로 나누고 집계함수를 적용해 결과를 합쳐주는 함수
```python
# Series 형식으로 뽑고 싶을 때
data.groupby(key_column)[value_column].function()
# Dataframe 형식으로 뽑고 싶을 때
data.groupby(key_column)[[value_column]].function()
data.groupby(key_column)[value_column].function().reset_index()
```

In [ ]:
# 데이터 불러오기
buy_info = pd.read_csv('Buy_info.csv', encoding = 'cp949')
id_info = pd.read_csv('Id_info.csv', encoding = 'cp949')
data = pd.merge(buy_info, id_info, on = 'ID')
data

In [ ]:
# column을 뽑지 않고 함수를 사용하면 문자열인 column을 제외하고 모든 열에 함수 적용
data.groupby('ID').sum()

In [ ]:
# column을 뽑고 함수를 사용하면 해당 열에 함수를 적용하고 이를 series로 반환
data.groupby('ID')['구매수량'].sum()

In [ ]:
# reset_index를 사용하면 기준열을 index에서 열로 바꾸고 dataframe 형식으로 반환
data.groupby('ID')['구매수량'].sum().reset_index()

In [ ]:
# 함수를 적용할 column을 복수로 추출 가능
data.groupby('ID')[['구매수량','상품대분류명']].nunique().reset_index()

In [ ]:
# 기준열 또한 복수 추출 가능, 먼저 쓴 열부터 기준
data.groupby(['ID','상품대분류명'])[['구매수량']].sum()

### 집계함수
- 집계함수는 기본적인 통계연산을 지원

 `mean(), std(), sum(), count(), max(), min(), size() 등`

- 직접 만든 함수를 사용하고 싶다면, agg() 메서드를 사용
```python
data.groupby(key_column)[value_column].agg()
```

In [ ]:
# agg()를 사용하여 복수의 함수 적용
data.groupby('ID')['구매수량'].agg(['mean','count'])

In [ ]:
# agg()안에는 일반적인 함수가 모두 들어갈 수 있음
data.groupby('ID')['구매수량'].agg(lambda x : x.max()- x.min()).reset_index()

In [ ]:
# 여러 함수를 적용할 때 column의 이름을 정해줄 수 있음
data.groupby('ID')['구매수량'].agg([('최대값','max'),('최소값','min')]).reset_index()

In [ ]:
# column을 뽑아오지 않고 각각의 column에 다른 함수를 적용 가능
data.groupby('ID').agg({'구매수량' : 'mean', '구매금액' : 'max'}).reset_index()

<font color = 'red'>Live Coding<p>
    <font color = 'black'>아래 사진과 같은 결과가 나오도록 하시오. (단, 구매변동은 표준편차입니다.)<p>
<img align="left" src = "ab.jpg" width = 350>

In [ ]:
# code here


# Pivot Table
<br><img align="left" src="http://drive.google.com/uc?export=view&id=1HEBp4qq4GaksdQBb2fx2tRsq-gVR-j6d" width=600>

- 데이터를 바탕으로, 서로 다른 column의 조합을 표로 만들어서 데이터를 일목요연하게 보기 위한 도구
- 피벗테이블을 잘 활용하면, 데이터를 상황에 맞춰 내가 원하는 조합으로 결합해 한눈에 볼 수 있음

- 피벗테이블의 기본형식
```python
pd.pivot_table(data, values = column_name, index = column_name, 
                 columns = column_name, aggfunc = function)
```

In [ ]:
# 사용자별 각 상품중분류명의 총구매금액
pd.pivot_table(data, values = '구매금액', index = 'ID', columns = '상품중분류명', aggfunc = np.sum).reset_index()

In [ ]:
# 사용자별 각 상품중분류명의 총구매액 (단, Nan => 0)
pd.pivot_table(data, values = '구매금액', index = 'ID', columns = '상품중분류명', aggfunc = np.sum, fill_value = 0).reset_index()

<font color = 'red'>Live Coding<p>
    <font color = 'black'>고객별 각 구매지역의 총구매수

In [ ]:
# code here


# Apply와 Applymap

- apply는 dataframe의 row 또는 column들에게 똑같은 function을 일괄적으로 적용해주는 함수
```python
dataframe.apply(function)
```

- applymap은 dataframe의 모든 요소에 같은 function을 일괄적으로 적용해주는 함수
```python
dataframe.applymap(function)
```

* 주로 lambda와 함께 사용

In [ ]:
# 1,2,3,4,5 그래프
pd.Series(range(1,5)).plot()
plt.show()

In [ ]:
# apply로 역수를 취한 그래프
pd.Series(range(1,5)).apply(lambda x: 1/x).plot()
plt.show()

In [ ]:
# apply를 활용해 column의 모든 값에 2 더해주기
start = t.time()

data['구매수량'].apply(lambda x : x+2).reset_index()

print(t.time()-start)

In [ ]:
# for문을 활용해 column의 모든 값에 2 더해주기
start = t.time()

구매수량_plus_2 = []
for i in data['구매수량']:
    구매수량_plus_2.append(i+2)
data['구매수량+2'] = 구매수량_plus_2

print(t.time()-start)

In [ ]:
# applymap을 활용해 dataframe의 모든 값에 2 곱하기
data2 = data[['ID','구매일자','구매시간','구매수량','구매금액']]
data2.applymap(lambda x : x*2)

# Datetime

In [ ]:
# datetime으로 자료형 바꿔주기
data['구매일자'] = pd.to_datetime(data.구매일자, format = '%Y%m%d')

In [ ]:
data.info()

In [ ]:
# datetime 함수
data['구매일자'].dt.year

In [ ]:
data['구매일자'].dt.month

In [ ]:
data['구매일자'].dt.day

In [ ]:
data['구매일자'].dt.dayofweek

In [ ]:
data['구매일자'].dt.day_name()